In [ ]:
# Import the necessary modules

import keras
from tensorflow.keras import datasets
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Activation 
from tensorflow.keras.layers import PReLU, LeakyReLU, Conv2D, Lambda, MaxPooling2D
from tensorflow.keras.regularizers import l2

from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras.models import model_from_json

from IPython.display import clear_output

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.ticker import MaxNLocator

import pickle
import sklearn as skl

from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score

In [ ]:
# Define some useful functions
class PlotLossAccuracy(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.acc = []
        self.losses = []
        self.val_losses = []
        self.val_acc = []
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(int(self.i))
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('accuracy'))
        self.val_acc.append(logs.get('val_accuracy'))
        
        self.i += 1
        
        clear_output(wait=True)
        plt.figure(figsize=(16, 6))
        plt.plot([1, 2])
        plt.subplot(121) 
        plt.plot(self.x, self.losses, label="train loss")
        plt.plot(self.x, self.val_losses, label="validation loss")
        plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.title('Model Loss')
        plt.legend()
        plt.subplot(122)         
        plt.plot(self.x, self.acc, label="training accuracy")
        plt.plot(self.x, self.val_acc, label="validation accuracy")
        plt.legend()
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.title('Model Accuracy')
        plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))
        plt.show();
        
def save_top_model_to_disk():    
    # save model and weights (don't change the filenames)
    model_json = top_model.to_json()
    with open("top_model.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    top_model.save_weights("top_model.h5")
    print("Saved top_model to top_model.json and weights to top_model.h5")

def save_finetuned_model_to_disk():    
    # save model and weights (don't change the filenames)
    model_json = finetuned_model.to_json()
    with open("finetuned_model.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    finetuned_model.save_weights("finetuned_model.h5")
    print("Saved model to finetuned_model.json and weights to finetuned_model.h5")



In [ ]:
!curl --create-dirs -o /home/tcd/data/cifar100-dataset.pkl http://4c16.ml/~mindfreeze/cifar100-dataset.pkl

In [ ]:
# Loading new dataset cifar100

print('loading the dataset...')

pkl_file = open('/home/tcd/data/cifar100-dataset.pkl', 'rb')
dataset = pickle.load(pkl_file)

print('loaded.')

print('let\'s look at some of the pictures and their ground truth labels:')

plt.figure(figsize=(12,12))
plt.plot([3, 3])

X = dataset['X'].astype('float32')/255
Y = dataset['Y'].astype('float32')
Y = keras.utils.to_categorical(Y)

for i in range(0,9):
    # pictures are 32x32x3 (width=32, height=32, 3 colour channels)
    pic = X[i]

    # Y[i] returns an array of zeros and with Y[i][classid] = 1
    # for instance  Y[i] = [ 0 0 0 0 0 1 0 0 0 0] => classid=5 
    #          and  Y[i] = [ 1 0 0 0 0 0 0 0 0 0] => classid=0
    # we can get the classid by using the argmax function on the vector Y[i]
    classid = Y[i].argmax(-1)

    # getting back the name of the label for that classid
    classname = dataset['labels'][classid]
    plt.subplot(3,3,i+1)
    plt.imshow(pic)
    plt.title('label: {}'.format(classname))


In [ ]:
# training / validation split
X_train, X_validation, Y_train, Y_validation = skl.model_selection.train_test_split(X, Y, test_size=.1)

In [ ]:
# Here we assume that you have already successfully trained a CNN on cifar10

# load json and create base_model
with open('../model.json', 'r') as json_file:
    loaded_model_json = json_file.read()
base_model = model_from_json(loaded_model_json)
# load weights into base_model
base_model.load_weights("../model.h5")
# print the model summary

In [ ]:
base_model.summary()

# In the summary, take note of the Layer names in the first column
# eg. conv2d_6 (Conv2D) means that 'conv2d_6' will be the name for 
# that layer

# In preparation for the next step, You also need to check that 
# shape of the Flatten layer is small enough. Typically we want something
# like (None, 2048). You don't want (None, 40000).

In [ ]:
# change the `flatten_2` name to the corresponding name from your network

# TODO: create model to predict the features at 'flatten' layer

new_model = keras.Model(inputs = base_model.get_layer('input_2').input, outputs = base_model.get_layer('flatten_1').output)
train_features = new_model.predict(X_train)
validation_features = new_model.predict(X_validation)

# checking that everything is ok, you should get a tensor of size 4500 x Flatten Size
# eg. (4500, 2048)
print(train_features.shape)

In [ ]:
# now that you have computed the new features for your dataset,
# you need to design a simple classification network.
# For good measure, use one with BatchNormalization, dropout, etc. 
# and train it.


inputs = keras.layers.Input(shape=train_features.shape[1:]) 
x = inputs

x = Dense(600, activation = 'relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)
x = Dense(500, activation = 'relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Dense(500, activation = 'relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

# TODO: complete the network ....
predictions = Dense(10, activation='softmax')(x)



In [ ]:
# now we can compile it and train it.

top_model = keras.models.Model(inputs=inputs, outputs=predictions)

# TODO: write code to train it ...

top_model = keras.models.Model(inputs=inputs, outputs=predictions)
opt = keras.optimizers.Adam(learning_rate=0.0005)
#, decay=1e-6, momentum=0.9, nesterov=True

# setup the optimisation strategy
top_model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


top_model.summary()

if (top_model.count_params() > 10000000):    
    raise("Your model is unecessarily complex, scale down!")

# the dataset is very small and the base_model gives us a very good 
# starting point, so we only need a handful of epochs, probably 1 
# or 2 will do.

pltCallBack = PlotLossAccuracy()

# and train
top_model.fit(train_features, Y_train,
          batch_size=1024, epochs=15, 
          validation_data=(validation_features, Y_validation), 
          callbacks=[pltCallBack])

In [ ]:
finetuned_model = keras.models.clone_model(base_model)

# set the all the layers in model to trainable = False

for layer in finetuned_model.layers:
    layer.trainable = False
    
# TODO: then set all the layers after the Flatten layer to be trainable 
finetuned_model.layers[12].trainable = True
finetuned_model.layers[13].trainable = True
finetuned_model.layers[14].trainable = True
finetuned_model.layers[15].trainable = True
finetuned_model.layers[16].trainable = True

In [ ]:
# TODO compile model 

# finetuned_model.compile(...)
finetuned_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
finetuned_model.summary()
# finetuned_model.summary(...)

# loading the pre-trained weights from your lab-05 network
finetuned_model.load_weights("../model.h5")

In [ ]:
# TODO train 'model' on (X_train, Y_train) for 10 or so epochs  
# What you should be able to observe is that the training is a bit slower than
# in the previous case. This is partly because we haven't re-initialised the weights.
num_epochs = 14

pltCallBack = PlotLossAccuracy()

finetuned_model.fit(X_train, Y_train,
          batch_size=1024, epochs=num_epochs, 
          validation_data=(X_validation, Y_validation), 
          callbacks=[pltCallBack])